In [28]:
import cv2
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

In [29]:
tree = ET.parse('train/hefei_3188.xml')
root = tree.getroot()

In [37]:
# 提取对象和边界框数据
data = []
for obj in root.findall('object'):
    name = obj.find('name').text
    bndbox = obj.find('bndbox')
    xmin = int(bndbox.find('xmin').text)
    ymin = int(bndbox.find('ymin').text)
    xmax = int(bndbox.find('xmax').text)
    ymax = int(bndbox.find('ymax').text)
    data.append([name, xmin, ymin, xmax, ymax])

# 转换为 DataFrame
df = pd.DataFrame(data, columns=['name', 'xmin', 'ymin', 'xmax', 'ymax'])

xmin = df['xmin'][0]
xmax = df['xmax'][0]
ymin = df['ymin'][0]
ymax = df['ymax'][0]

xmax - xmin

19

In [38]:
image_dirname = 'hefei_3188.jpg'
image_path = 'images/' + image_dirname
image = cv2.imread(image_path, cv2.IMREAD_COLOR)
image.shape

(296, 400, 3)

In [39]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# 中值滤波去噪
gray = cv2.medianBlur(gray, 5)

In [40]:
points = np.array([[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax]], dtype='float32')
width = xmax - xmin
height = ymax - ymin
dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype='float32')

In [41]:
# 去照光
adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 11, 2)

In [42]:
# 计算透视变换矩阵
matrix = cv2.getPerspectiveTransform(points, dst_points)

# 进行透视变换
warped = cv2.warpPerspective(adaptive_thresh, matrix, (width, height))

res_image = cv2.resize(warped,(20,32))

solved_image = np.expand_dims(res_image, axis=-1)

solved_image.shape

(32, 20, 1)

In [43]:
cv2.imshow('Processed Image', solved_image)
cv2.waitKey(0)
cv2.destroyAllWindows()